In [ ]:
# This notebook uses sympy to calculate the appropriate coefficients 
# for the derivative stencils for the various cases used in engrenage
# NB The logarithmic case isn't used any more but is useful for reference 

# restart the kernel to clear past work
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [6]:
import numpy as np
from sympy import symbols, simplify, pprint, diff, expand, collect, finite_diff_weights, apply_finite_diff
import matplotlib.pyplot as plt
import sys
%matplotlib inline

sys.path.append('../')
from source.grid import Grid                   # go here to see how the grid and derivatives are calculated
from source.spacing import *                   # go here to import spacing

In [5]:
# Check the finite derivatives for a fixed spacing dx, centered derivative
# using 5 points (so O(dx^4) accurate)

# Construct the Lagrange polynomial using sympy
x = symbols('x')
deltax = symbols('dx')
x3 = symbols('x3')
x2 = x3-deltax
x1 = x2-deltax
x4 = x3+deltax
x5 = x4+deltax
f1 = symbols('f1')
f2 = symbols('f2')
f3 = symbols('f3')
f4 = symbols('f4')
f5 = symbols('f5')

L1 = (x - x2 ) / (x1 - x2) * (x - x3 ) / (x1 - x3)* (x - x4) / (x1 - x4) * (x - x5 ) / (x1 - x5)
L2 = (x - x1 ) / (x2 - x1) * (x - x3 ) / (x2 - x3)* (x - x4 ) / (x2 - x4)* (x - x5 ) / (x2 - x5)
L3 = (x - x1 ) / (x3 - x1) * (x - x2 ) / (x3 - x2)* (x - x4 ) / (x3 - x4)* (x - x5 ) / (x3 - x5)
L4 = (x - x1 ) / (x4 - x1) * (x - x2 ) / (x4 - x2)* (x - x3 ) / (x4 - x3)* (x - x5 ) / (x4 - x5)
L5 = (x - x1 ) / (x5 - x1) * (x - x2 ) / (x5 - x2)* (x - x3 ) / (x5 - x3)* (x - x4 ) / (x5 - x4)
P = f1 * L1 + f2 * L2 + f3 * L3 + f4 * L4 + f5 * L5
#print("Using sympy we get the polynomial", simplify(P))
#pprint(simplify(P))

# Now take the derivative twice
dPdx = diff(P, x)
#print("Its first derivative is", simplify(dPdx))

d2Pdx2 = diff(dPdx, x)
#print("Its second derivative is", simplify(d2Pdx2))
#pprint(simplify(dPdx))

# Now find the value at the central point x3
dPdx_at_x3 = dPdx.subs(x, x3)
d2Pdx2_at_x3 = d2Pdx2.subs(x, x3)
print("Its first derivative at x3 is", simplify(dPdx_at_x3))
print("Its second derivative at x3 is", simplify(d2Pdx2_at_x3))
print("This gives the first derivative stencil \n")
pprint(simplify(dPdx_at_x3))
print("This gives the second derivative stencil \n")
pprint(simplify(d2Pdx2_at_x3))

Its first derivative at x3 is (f1 - 8*f2 + 8*f4 - f5)/(12*dx)
Its second derivative at x3 is (-f1 + 16*f2 - 30*f3 + 16*f4 - f5)/(12*dx**2)
This gives the first derivative stencil 

f₁ - 8⋅f₂ + 8⋅f₄ - f₅
─────────────────────
        12⋅dx        
This gives the second derivative stencil 

-f₁ + 16⋅f₂ - 30⋅f₃ + 16⋅f₄ - f₅
────────────────────────────────
                  2             
             12⋅dx              


In [6]:
# Example with non even spacing - the logarithmic case

# Check the finite derivatives for log factor of c, centered derivative
# using 5 points (so O(dx^4) accurate)

# Construct the polynomial using sympy
x = symbols('x')
c = symbols('c')
dx = symbols('dx')
x1 = dx/2
x2 = x1+(c*dx)
x3 = x2+(c*c*dx)
x4 = x3+(c*c*c*dx)
x5 = x4+(c*c*c*c*dx)
f1 = symbols('f1')
f2 = symbols('f2')
f3 = symbols('f3')
f4 = symbols('f4')
f5 = symbols('f5')

L1 = (x - x2 ) / (x1 - x2) * (x - x3 ) / (x1 - x3)* (x - x4) / (x1 - x4) * (x - x5 ) / (x1 - x5)
L2 = (x - x1 ) / (x2 - x1) * (x - x3 ) / (x2 - x3)* (x - x4 ) / (x2 - x4)* (x - x5 ) / (x2 - x5)
L3 = (x - x1 ) / (x3 - x1) * (x - x2 ) / (x3 - x2)* (x - x4 ) / (x3 - x4)* (x - x5 ) / (x3 - x5)
L4 = (x - x1 ) / (x4 - x1) * (x - x2 ) / (x4 - x2)* (x - x3 ) / (x4 - x3)* (x - x5 ) / (x4 - x5)
L5 = (x - x1 ) / (x5 - x1) * (x - x2 ) / (x5 - x2)* (x - x3 ) / (x5 - x3)* (x - x4 ) / (x5 - x4)
P = f1 * L1 + f2 * L2 + f3 * L3 + f4 * L4 + f5 * L5
#print("Using sympy we get the polynomial", simplify(P))
#pprint(simplify(P))

# Now take the derivative twice
dPdx = diff(P, x)
#print("Its first derivative is", simplify(dPdx))

d2Pdx2 = diff(dPdx, x)
#print("Its second derivative is", simplify(d2Pdx2))
#pprint(simplify(dPdx))

# Now find the value at the central point x3
dPdx_at_x3 = dPdx.subs(x, x3)
d2Pdx2_at_x3 = d2Pdx2.subs(x, x3)
#print("Its first derivative at x2 is", simplify(dPdx_at_x3))
#print("Its second derivative at x2 is", simplify(d2Pdx2_at_x3))
#print("This gives the first derivative stencil \n")
#pprint(simplify(dPdx_at_x3))
#print("This gives the second derivative stencil \n")
#pprint(simplify(d2Pdx2_at_x3))

# Easier to isolate out the terms one by one - coeff of f5 first
f5_dPdx_at_x3 = dPdx_at_x3.subs(f1, 0)
f5_dPdx_at_x3 = f5_dPdx_at_x3.subs(f2, 0)
f5_dPdx_at_x3 = f5_dPdx_at_x3.subs(f3, 0)
f5_dPdx_at_x3 = f5_dPdx_at_x3.subs(f4, 0)
f5_dPdx_at_x3 = simplify(f5_dPdx_at_x3)
dR_at_x3 = (c*c*dx)
print("The coefficient of the f5 term in the first derivative stencil is \n")
pprint(f5_dPdx_at_x3 * dR_at_x3)

print("\n In the case where c is 1 we can check it matches the above:")
pprint(simplify(((dPdx_at_x3 * dR_at_x3).subs(c,1))))

The coefficient of the f5 term in the first derivative stencil is 

               -f₅               
─────────────────────────────────
 2 ⎛ 2        ⎞ ⎛ 3    2        ⎞
c ⋅⎝c  + c + 1⎠⋅⎝c  + c  + c + 1⎠

 In the case where c is 1 we can check it matches the above:
f₁   2⋅f₂   2⋅f₄   f₅
── - ──── + ──── - ──
12    3      3     12


In [7]:
# check the form matches the simpler one used in the code
# (sympy not great at simplifying)
check = -f5/(c**2 * (1 + c) *(1 + c**2) * (1 + c + c**2))
simplify(f5_dPdx_at_x3 * dR_at_x3 - check)

0

In [8]:
# Same for second derivs - coeff of f5 first
f5_d2Pdx2_at_x3 = d2Pdx2_at_x3.subs(f1, 0)
f5_d2Pdx2_at_x3 = f5_d2Pdx2_at_x3.subs(f2, 0)
f5_d2Pdx2_at_x3 = f5_d2Pdx2_at_x3.subs(f3, 0)
f5_d2Pdx2_at_x3 = f5_d2Pdx2_at_x3.subs(f4, 0)
f5_d2Pdx2_at_x3 = simplify(f5_d2Pdx2_at_x3)
dR_at_x3 = (c*c*dx)
print("The coefficient of the f5 term in the first derivative stencil is \n")
pprint(simplify(f5_d2Pdx2_at_x3 * dR_at_x3 * dR_at_x3))

The coefficient of the f5 term in the first derivative stencil is 

           ⎛ 2                    ⎞      
     -2⋅f₅⋅⎝c  + c⋅(c + 1) - c - 1⎠      
─────────────────────────────────────────
 3         ⎛ 2        ⎞ ⎛ 3    2        ⎞
c ⋅(c + 1)⋅⎝c  + c + 1⎠⋅⎝c  + c  + c + 1⎠
